In [25]:

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from joblib import load
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

In [30]:
X_train,y_train = split_data.get_train_data()

In [31]:
X_test,y_test = split_data.get_test_data()

In [16]:


# Define the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', load('PreprocessingPipeline.pkl')),
    ('xgboost', XGBClassifier()),

    
])

# Fit the model pipeline to the training data
model_pipeline.fit(X_train, y_train)
predict = model_pipeline.predict(X_test)
print(recall_score(predict,y_test))


0.9958289885297185


In [18]:
loaded_pipeline = joblib.load('preprocessing_pipeline.joblib')
X_processed_data = loaded_pipeline.transform(X_train)

# group / ensemble of models
estimator = []
estimator.append(('LR', LogisticRegression()))
estimator.append(('SVC', SVC(kernel='linear')))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('RFC', RandomForestClassifier()))
  
  
# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_processed_data, y_train)
y_pred = vot_hard.predict(loaded_pipeline.transform(X_test))

In [19]:
print(recall_score(y_pred,y_test))

1.0


In [16]:
# Load the pipeline from disk using joblib
loaded_pipeline = joblib.load('preprocessing_pipeline.joblib')
X_processed_data = loaded_pipeline.transform(X_train)


# fit model no training data
model_xgb = XGBClassifier()
model_xgb.fit(X_processed_data, y_train)

In [9]:
#!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
## Pending!!!!!


# rskf = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 2, random_state = 42)

In [35]:
y_pred_xgb = model_xgb.predict(loaded_pipeline.transform(X_test))

In [36]:
print(recall_score(y_pred_xgb,y_test))

0.9944588104913188


In [26]:
# !pip install import-ipynb


In [27]:

print(processed_data)

(       Unnamed: 0   Time CellName  PRBUsageUL  PRBUsageDL  meanThr_DL  \
0           35217   8:30    9ALTE       3.537       0.808       0.148   
1             507  18:15    4CLTE      15.259       1.819       0.457   
2            6074   7:15    6ALTE       3.335       0.909       0.448   
3           12206  21:30    7WLTE       2.728       5.154       0.786   
4           29773  18:45    6ULTE       2.526       5.558       1.936   
...           ...    ...      ...         ...         ...         ...   
25827       32081  19:45    6BLTE      24.875       2.786       0.803   
25828        7259  15:30    3CLTE      11.217       2.829       1.018   
25829       21584  20:45    5ALTE       6.164       0.808       0.279   
25830       36543  19:30    4ALTE       7.882       1.011       0.203   
25831       10137   3:45    3BLTE       4.080       2.090       0.911   

       meanThr_UL  maxThr_DL  maxThr_UL  meanUE_DL  meanUE_UL  maxUE_DL  \
0           0.013      4.236      0.111      1.

In [68]:

pipeline = Pipeline([
    ('preprocessing', load('pipelines/PreprocessingPipeline.pkl'))
])
data = pd.read_csv('../data/raw/data.csv',  encoding= 'unicode_escape')
data = data.drop(["Unusual"], axis=1)
X_transformed = pipeline.fit_transform(data)
    
#feature_names = x_data.named_transformers_['encoder'].get_feature_names_out()






# Get the output feature names

#column_transformer = ColumnTransformer([
#    ('preprocessing_pipeline', preprocessing_pipeline)
#])

#feature_names = column_transformer.named_transformers['cat'] \
#                                .named_steps['encoder'] \
#                                .get_feature_names_out()
# Fit and transform the data using the pipeline
#transformed_data = pipeline.fit_transform(data)
#pd.DataFrame(pipeline.fit_transform(data)).to_csv(test.csv, index=False)

ValueError: A given column is not a column of the dataframe